# **Challenge - Alkemy**

# **Parte 1 - Requerimiento Funcional - Archivos Fuentes**
1) Para descargar las tablas desde el sitio web debemos tener instalada la librería "requests"
    - Windows: debemos tener instalado python. Luego, abrir una terminal y escribir lo siguiente: python -m pip install requests
    - Linux: desde la terminal: pip install requests
2) Importamos las librerias a utilizar
3) Cargamos las funciones que vamos a utilizar
4) Ejecutamos

## **1) Instalamos librerias**

In [18]:
# En Linux:
#!pip install requests
#!pip install logging

## **2) Importamos librerías a utilizar:**

In [19]:
import requests   
import re    # Regular Expression module: re.search, findall, 
import os    # path.join
import sys   # sys.exit()
from datetime import datetime, date
import logging

# Seteamos el idioma español para las fechas (funciones: fecha_ultimo_cambio y file_path)
import locale
locale.setlocale(locale.LC_ALL,'es_ES.UTF-8')

'es_ES.UTF-8'

## **3) Funciones a Utilizar**

In [20]:
def requests_get_OK(web):
    ''' Verifica el código de estado de la respuesta de requests.get
        
    INPUT: web: es objeto Response que se obtiene al ejecutar: requests.get(url)
    OUTPUT: código del estado del request
    '''
    codigo = web.status_code   # si el valor es 200 significa que funcionó el "requests.get"
    if  codigo == 200:
        logging.info(f'Requests Exitoso del link.')   # si el link es el del archivo, saco el nombre de ahí con split
    elif codigo == 504:
        logging.warning(f'Falla del Servidor. Error: {codigo} Gateway Time-out')
        sys.exit(f'Falla del Servidor. Vuelva a intentar. Error: {codigo} Gateway Time-out')
    else:
        logging.warning(f'Falló el Requests. Error: {codigo}') 
        sys.exit(f'Falló el Requests. Error: {codigo}')
    
    return

In [21]:
def obtener_link(url_pagina):
    ''' Verifica que existe la URL y obtiene el link de descarga de la tabla de datos.
    Controla que la pagina tenga el botón "DESCARGAR" que se usa para descargar el archivo.
        
    INPUT: url_pagina: enlace de la página web de la cual queremos obtener el link de descarga.
    OUPUT: link_descarga: enlace de descarga
    '''
    
    logging.info(f'__Pagina web a utilizar: {url_pagina}')
    
    # Entra y verifica la pagina
    pagina = requests.get(url = url_pagina)
    requests_get_OK(pagina)
    
    # Busca y Extrae el link
    inicio = 'href="'
    final = '">DESCARGAR'
    link_descarga = re.search(f'{inicio}(.*?){final}', pagina.text)[1]
    
    if link_descarga: 
        logging.info(f'Se ha obtenido el link de descarga: {link_descarga}')
        return link_descarga
    else:
        logging.warning(f'No se encuentra el link de descarga: {link_descarga}')
        return print('No se encuentra el link de descarga')
        
    

In [22]:
def descargar_tabla(url_descarga):
    ''' Descarga la tabla de datos en formato binario
    
    INPUT: url_descarga: enlace de descarga del archivo con los datos (archivo .csv).
    OUPUT: contenido: tabla en formato binario.    
    '''
    
    pagina = requests.get(url = url_descarga)
    requests_get_OK(pagina)
    
#     print(pagina.apparent_encoding)

    # Extrae el contenido del archivo
    contenido = pagina.content   # Contains bytes with the raw response content.
    
    file_name = url_descarga.split("/")[-1]  # nombre del archivo de la url de descarga
    logging.info(f'Se ha descargado la información de {file_name}')

    return contenido

In [23]:
def file_path(categoria_link):
    ''' Crea la estructura de carpetas donde se guardarán las tablas.
    Por ej: Cultura_dataset/museos/2021-diciembre/museos-10-12-2021.csv
    
    INPUT: categoria_link: link de descarga de la tabla de datos
    OUTPUT: file_path: el path completo donde se guardará el archivo
    '''
#     meses_traduccion = {'January':'Enero', 'February':'Febrero', 'March':'Marzo',
#                     'April':'Abril', 'May':'Mayo', 'June':'Junio',
#                     'July':'Julio', 'August':'Agosto', 'September':'Septiembre',
#                     'October':'Octubre', 'November':'Noviembre', 'December':'Diciembre' }
    
    # Carpeta principal
    path = 'Cultura_dataset'
    
    # Carpeta secundaria para la categoría correspondiente
    file_name = categoria_link.split('/')[-1]
    folder_category = file_name.replace('.csv','')
    
    # Carpeta terciaria 
    year = date.today().strftime('%Y')
    month = date.today().strftime('%B')
#     if month in meses_traduccion:   #Si el mes está en inglés, lo traducimos
#         month = meses_traduccion[month].lower()   # mes en minúsculas
#     else:
#         month = month.lower()
    folder_date = year+'-'+month  # YYYY-mes
    
    # Path final para la categoría correspondiente
    path_final = os.path.join(path,folder_category, folder_date)

    # Creamos las carpetas de forma recursiva si NO existen (exist_ok=True).
    os.makedirs(path_final, exist_ok=True)
    
    # # Fecha de Descarga DD-MM-YYYY                      ----  ? ? ? ? ? ? ? ----
    download_date = date.today().strftime('%d-%m-%Y')  
    # Nombre del archivo a guardar
    file = folder_category + '-' + download_date + '.csv'   
    # Path con el nombre del archivo a guardar
    file_path = os.path.join(path_final, file)
    
    logging.info(f'El path completo con el archivo es: {file_path}')
    
    return file_path

In [24]:
def guardar_archivo(file_path, categoria_tabla):
    ''' Crea el archivo y guarda la tabla de datos en el mismo.
    
    INPUT: file_path: El path donde se guardarán los datos
           categoria_tabla: variable que contiene el contenido de la tabla
    OUTPUT:
    '''        
    csv_file = open(file_path, 'wb')  # For text responses you need to use "w" - plain write mode. For binary "wb"
    csv_file.write(categoria_tabla)
    csv_file.close()
    
    logging.info(f'Se guardó el archivo en: {file_path}')

    return print(f'Se guardó el archivo en: {file_path}')

## **4) Ejecutamos**

In [25]:
''' URLs dadas:
No es la url directa a la tabla, sino que tenemos que buscarla.'''

url_museos = 'https://datos.gob.ar/dataset/cultura-mapa-cultural-espacios-culturales/archivo/cultura_4207def0-2ff7-41d5-9095-d42ae8207a5d'
url_cines = 'https://datos.gob.ar/dataset/cultura-mapa-cultural-espacios-culturales/archivo/cultura_392ce1a8-ef11-4776-b280-6f1c7fae16ae'
url_bibliotecas = 'https://datos.gob.ar/dataset/cultura-mapa-cultural-espacios-culturales/archivo/cultura_01c6c048-dbeb-44e0-8efa-6944f73715d7'

In [26]:
urls = ['url_museos', 'url_cines', 'url_bibliotecas']

In [27]:
# Creamos el archivo info.LOG 
logging.basicConfig(filename= 'info.log', format='%(asctime)s : %(levelname)s : %(message)s',
                    datefmt='%d/%m/%y %H:%M:%S', level=logging.INFO,
                    filemode = 'w', force=True)
logging.info('EJECUTANDO LA PARTE 1 (1er archivo.... ')

# Ejecutamos las funciones para descargar las tablas de datos
for site in urls:
    print(site)
    link = obtener_link(eval(site))
    
    tabla = descargar_tabla(link)
    
    path = file_path(link)    
    guardar_archivo(path, tabla)

logging.info('.........Terminó la descarga de las tablas (1ra Parte) ')
print('\n.........Terminó la descarga de las tablas (1ra Parte) ')

url_museos
Se guardó el archivo en: Cultura_dataset/museos/2022-enero/museos-06-01-2022.csv
url_cines
Se guardó el archivo en: Cultura_dataset/cine/2022-enero/cine-06-01-2022.csv
url_bibliotecas
Se guardó el archivo en: Cultura_dataset/biblioteca_popular/2022-enero/biblioteca_popular-06-01-2022.csv

.........Terminó la descarga de las tablas (1ra Parte) 


**LOGGING no me sobreescribe el archivo aunque use 'w'. Debe ser porque lo defino fuera de funciones y lo uso dentro de funciones**

## **Pruebas**

In [28]:
# p = url_cines
# pagina = requests.get(url = p)

# # Busca y Extrae el link
# inicio = 'href="'
# final = '">DESCARGAR'
# link_descarga = re.search(f'{inicio}(.*?){final}', pagina.text)[1]
# print(link_descarga)

# tab = requests.get(url = link_descarga)
# print(tab.status_code)

# encod = tab.apparent_encoding
# print('encod= ', encod)

In [29]:
# import csv
# aux = 'cin.csv'
# # with open(aux, "w") as file:   #
# #     file.write(tab.text)

# csv_file = open(aux, 'w', encoding = encod)  # wb: Sobreescribe si existe, sino lo crea. , 
# csv_file.write(tab.text)
# csv_file.close()

In [30]:
# !pwd
# import chardet
# with open(aux, 'rb') as rawdata:
#     result = chardet.detect(rawdata.read(1000000))
# print(result)

In [31]:
# import pandas as pd
# df = pd.read_csv(aux)  # (utf-8 da error con "museos" nada más) --  ( latin-1  funciona )
# df.head(2)

In [32]:
# Creamos el archivo info.LOG 
# logging.basicConfig(filename= 'info.log', format='%(asctime)s : %(levelname)s : %(message)s',
#                     datefmt='%d/%m/%y %H:%M:%S', level=logging.INFO,
#                     filemode = 'w', force=True)